<a href="https://colab.research.google.com/github/Miiidou/finger_number/blob/main/finger_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Réseau neuronal permettant d'indentifier le chiffre indiqué avec les doigts. 

import cv2
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import datetime
import zipfile
import pathlib
import os
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras import layers

%pylab inline

# Import training dataset
data_dir = tf.keras.utils.get_file(
    "dataset.zip",
    "https://github.com/Miiidou/finger_number/blob/3e1ab0b81896510c835c6a7b7fd3f9400ec80f20/dataset.zip?raw=true",
    extract=False)

with zipfile.ZipFile(data_dir, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

data_dir = pathlib.Path('/content/datasets/dataset')

# Import validation dataset
validation_dir = tf.keras.utils.get_file(
    "test.zip",
    "https://github.com/Miiidou/finger_number/blob/35d939e625399f8c11c6406758ed481ad3fdd01b/test.zip?raw=true",
    extract=False)

with zipfile.ZipFile(validation_dir, 'r') as zip_ref:
    zip_ref.extractall('/content/datasets')

validation_dir = pathlib.Path('/content/datasets/test')

# Convert JPG files to PNG
from PIL import Image
datasetsList = [pathlib.Path(data_dir).glob('*'),pathlib.Path(validation_dir).glob('*')]
for idx, dataset in enumerate(datasetsList) :
  for data_folder in dataset :
    folders = [f for f in listdir(data_folder) if isfile(join(data_folder, f))]
    for fichier in folders : 
      file_name = pathlib.Path(fichier).stem
      file_ext = pathlib.Path(fichier).suffix
      file_path_jpg = join(data_folder,fichier)
      file_path_png = pathlib.Path(str(data_folder) + "/" + str(file_name) + ".png")
      if((file_ext == ".jpg") and (not isfile(file_path_png))) :
        im = Image.open(str(file_path_jpg))
        im.save(str(file_path_png)) 
        os.remove(file_path_jpg) 
      elif(file_ext == ".jpg") : 
        os.remove(file_path_jpg) 
    folders = [f for f in listdir(data_folder) if isfile(join(data_folder, f))]
    if(idx == 0) :
      print("Training dataset " + str(data_folder.stem) + " - " + str(len(folders)) + " items")
    elif(idx == 1) :
      print("Validation dataset " + str(data_folder.stem) + " - " + str(len(folders)) + " items")

# Préprocessing 
batch_size = 3
img_height = 200
img_width = 200

train_data = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  )

val_data = tf.keras.preprocessing.image_dataset_from_directory(
  validation_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = val_data.class_names
# print(class_names)

# Neuronal network
num_classes = 6

model = tf.keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(128,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,4, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

logdir="logs"

tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_images=logdir,
                                                   embeddings_data=train_data)

model.fit( 
  train_data,
  validation_data=val_data,
  epochs=100,
  callbacks=[tensorboard_callback]
)

model.summary()

# Enjoying prediction
from google.colab import files
file_to_predict = files.upload()
for file_ in file_to_predict:
    image_to_predict = cv2.imread(file_,cv2.IMREAD_COLOR)
    plt.axis('off')
    plt.imshow(cv2.cvtColor(image_to_predict, cv2.COLOR_BGR2RGB))
    plt.show()
    img_to_predict = np.expand_dims(cv2.resize(image_to_predict,(200,200)), axis=0) 
    res = np.argmax(model.predict(img_to_predict), axis=1)
    print(model.predict(img_to_predict))
    if res == 0:
        print("IT'S A 0 !")
    elif res == 1 :
        print("IT'S A 1 !")
    elif res == 2 :
        print("IT'S A 2 !")
    elif res == 3 :
        print("IT'S A 3 !")
    elif res == 4 :
        print("IT'S A 4 !")
    elif res == 5 :
        print("IT'S A 5 !")

Populating the interactive namespace from numpy and matplotlib
Training dataset 5 - 25 items
Training dataset 1 - 25 items
Training dataset 3 - 25 items
Training dataset 0 - 25 items
Training dataset 4 - 25 items
Training dataset 2 - 25 items
Validation dataset 5 - 15 items
Validation dataset 1 - 3 items
Validation dataset 3 - 6 items
Validation dataset 0 - 8 items
Validation dataset 4 - 8 items
Validation dataset 2 - 10 items
Found 150 files belonging to 6 classes.
Using 120 files for training.
Found 50 files belonging to 6 classes.
Using 10 files for validation.


Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


40/40 [==============================] - 37s 908ms/step - loss: 1.8102 - accuracy: 0.1250 - val_loss: 1.7867 - val_accuracy: 0.3000
Epoch 2/100
40/40 [==============================] - 36s 892ms/step - loss: 1.7945 - accuracy: 0.1417 - val_loss: 1.7944 - val_accuracy: 0.2000
Epoch 3/100
40/40 [==============================] - 36s 899ms/step - loss: 1.7925 - accuracy: 0.1417 - val_loss: 1.7937 - val_accuracy: 0.1000
Epoch 4/100
40/40 [==============================] - 36s 888ms/step - loss: 1.7921 - accuracy: 0.1833 - val_loss: 1.7935 - val_accuracy: 0.1000
Epoch 5/100
40/40 [==============================] - 36s 891ms/step - loss: 1.7918 - accuracy: 0.1833 - val_loss: 1.7946 - val_accuracy: 0.1000
Epoch 6/100
40/40 [==============================] - 37s 904ms/step - loss: 1.8044 - accuracy: 0.2000 - val_loss: 1.7986 - val_accuracy: 0.1000
Epoch 7/100
40/40 [==============================] - 37s 909ms/step - loss: 1.7964 - accuracy: 0.1833 - val_loss: 1.7936 - val_accuracy: 0.1000
Epoc